Unfortunately, my NDA prevents me from sharing my dataset or the code I used to preprocess the data. I can only share the code I used for my Keras model.

In [1]:
# implements squeeze excitation. 
# copied from https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7
def se_block(in_block, ch = X_train.shape[-1], ratio=16):
    x = GlobalAveragePooling2D()(in_block)
    x = Dense(ch//ratio, activation='relu')(x)
    x = Dense(ch, activation='sigmoid')(x)
    return Multiply()([in_block, x])

NameError: name 'X_train' is not defined

In [2]:
# code adapted from Francois Chollet. see here: https://github.com/timrauen/deep-learning-models/blob/master/resnet50.py

def add_resid_layers(input_tensor, block, filters, layers = 2):
    X = input_tensor
    # Right now i'm only using blocks of 2 layers. maybe it would work better with larger blocks?
    for layer in range(layers):
      # adds a convolutional layer with batch normalization and dropout
      X = Conv2D(filters, (3, 3), padding = 'same', kernel_regularizer = l2(.001), 
                 name = (f'block_{block}_layer_{layer}'))(X)
      X = BatchNormalization(axis = 3)(X)
      X = Activation('relu')(X)
      X = Dropout(.5)(X)
    # If the number of filters in this block is different from the previous layer we must apply a (1,1)
    # convolution with the appropriate number of filters so the tensors can be added.
    if X.shape == input_tensor.shape:
      X = Add()([X, input_tensor])
      X = Activation('relu')(X)
    else:
      shortcut = Conv2D(filters, (1,1))(input_tensor)
      shortcut = BatchNormalization(axis = 3)(shortcut)
      X = Add()([X, shortcut])
    X = se_block(X, ch = filters)
    return X

In [3]:
def model_resid():
  input = Input((8, 8, X_train.shape[-1]))
  # start with 16 to force it to be selective about early local features
  # so the model would be selective learning local features
  X = Conv2D(16, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer = l2(.001))(input) 
  # Gradually increasing the number of filters.
  X = add_resid_layers(X, block = 1, filters = 32)
  X = add_resid_layers(X, block = 2, filters = 32)
  X = add_resid_layers(X, block = 3, filters = 64)
  X = add_resid_layers(X, block = 4, filters = 64)
  X = add_resid_layers(X, block = 5, filters = 64)
  X = add_resid_layers(X, block = 6, filters = 64)
  X = add_resid_layers(X, block = 7, filters = 128)
  X = add_resid_layers(X, block = 8, filters = 128)
  # Reducing the number of filters before flattening
  X = Conv2D(4, (3, 3), activation = 'relu', kernel_regularizer = l2(.001))(X) #16
  X = Flatten()(X)
  X = Dense(512, activation = 'relu')(X)
  X = Dense(1)(X)
  model = Model(inputs = input, outputs = X, name = 'residual_model')
  return model

In [4]:
model = model_resid()
model.compile(loss = 'mean_squared_error',
             optimizer = 'adam',
             metrics = 'mean_squared_error',
             )

NameError: name 'Input' is not defined

In [ ]:
early_stop = EarlyStopping(patience = 6)
history = model.fit(X_train, y_train, epochs=50, 
                    validation_data=(X_test, y_test),
                    callbacks = [early_stop])